# Réalisez un dashboard
## Notebook - Test de l'API  
OpenClassrooms - Parcours Data Scientist - Projet 08  

__Ce notebook teste l'API en ligne :__  
* Sélection d'un exemple dans le jeu de test  
* Interrogation de l'API  
* Affichage de la probabilité de défaut de remboursement
* Affichage de l'accord ou non du crédit
* Affichage des 20 éléments ayant eu le plus d'impact sur la prédiction

In [23]:
import pandas as pd
import numpy as np
import requests
import json
import os

In [24]:
# Nombre d'exemples à tester
n_samples = 1

In [25]:
# Récupération des données de test
base_dir = os.getcwd()
df_test_path = os.path.join(base_dir, '..', '..', 'P08 - API', 'data', 'raw', 'application_test.csv')
df_test = pd.read_csv(df_test_path)

In [26]:
# Récupération du seuil de classification
threshold_path = os.path.join(base_dir, '..', '..', 'P08 - API', 'data', 'processed', 'best_threshold.txt')
with open(threshold_path, 'r') as threshold_file:
    best_threshold = float(threshold_file.read())

In [27]:
# Sélection aléatoire de n_samples exemples
df_subset = df_test.sample(n_samples)

In [28]:
# Récupération des id clients
sk_id_curr = df_subset['SK_ID_CURR'].astype('int')

In [29]:
# Suppression des id clients du subset
df_subset = df_subset.drop(columns='SK_ID_CURR')

In [30]:
# Remplacement des valeurs manquantes par None (JSON n'accepte pas les NaN)
df_subset =  df_subset.map(lambda x: None if pd.isna(x) else x)

In [31]:
# Conversion au format JSON
data_json = json.dumps({
    # "columns": df_subset.columns.tolist(),
    "data": df_subset.values.tolist()}
)

In [32]:
# Envoi de la requête POST à l'API
heroku_url = 'https://failurescore-bc9f53f25e58.herokuapp.com/predict'
local_url = 'http://127.0.0.1:5000/predict'
response = requests.post(
    heroku_url,
    headers={'Content-Type': 'application/json'},
    data=data_json
)

In [33]:
# Vérification du code status
display(response)

<Response [500]>

In [34]:
df_subset

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
40852,Cash loans,F,N,Y,0,90000.0,327249.0,26365.5,292500.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0


In [ ]:
# DataFrame pour affichage des probabilités et accord ou non de crédit
predictions = pd.DataFrame({'Customer id': sk_id_curr.values.tolist(), 'Failure probability': response.json()['prediction_proba'], 'Credit agreement': response.json()['prediction_class']})
predictions['Credit agreement'] = predictions['Credit agreement'] == 0
predictions['Failure probability'] = round(predictions['Failure probability'], 2)

In [ ]:
# Affichage
display(predictions)

In [35]:
data_json

'{"data": [["Cash loans", "F", "N", "Y", 0, 90000.0, 327249.0, 26365.5, 292500.0, "Family", "Working", "Secondary / secondary special", "Married", "House / apartment", 0.014464, -16000, -3373, -9251.0, -5747, null, 1, 1, 0, 1, 0, 0, "Laborers", 2.0, 2, 2, "TUESDAY", 7, 0, 0, 0, 0, 0, 0, "Agriculture", 0.2774370278715383, 0.4960056678294886, null, 0.0062, null, 0.9677, null, null, null, null, 0.0, null, 0.0544, null, 0.004, null, null, 0.0063, null, 0.9677, null, null, null, null, 0.0, null, 0.0556, null, 0.0042, null, null, 0.0062, null, 0.9677, null, null, null, null, 0.0, null, 0.0553, null, 0.0041, null, null, null, "block of flats", 0.0031, "Monolithic", "No", 0.0, 0.0, 0.0, 0.0, 0.0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0]]}'